<h1>判例予測モデルの構築と検証</h1>

それぞれのカテゴリ変数のうち、メジャーな20種類（裁判官でいえば最もよく登場する20人）のみを採用し、他は欠損として扱っています。
<br>精度は75%程度に落ちましたが、極端にデータが少ない有罪・破棄・破棄差戻を除き、幅広い予測結果が得られるようになりました。

<h2>分析パラメータ設定</h2>

In [1]:
#目的変数
target = "trial_result"
#説明変数各種カテゴリ変数のうち、上位何種類を使うか
exnum = 20

<h2>データ読み込み</h2>

In [2]:
#pandasを使ってcsvファイルを読み込み
import pandas as pd
#pandasのwarningを非表示にする
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("query_result.csv")
#データ確認
df.head()

,id,case_name,case_type,court_name,trial_date,trial_type,trial_result,id.1,lawyer_name,office,id.2,judge_name
0,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,2006-01-19,NaN,取り消す,12013,高坂敬三,色川法律事務所,39.0,安西二郎
1,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,2006-01-19,NaN,取り消す,13064,永野周志,シード綜合法律事務所,39.0,安西二郎
2,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,2006-01-19,NaN,取り消す,20798,鳥山半六,色川法律事務所,39.0,安西二郎
3,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,2006-01-19,NaN,取り消す,24026,山崎徹,弁護士法人川越法律事務所,39.0,安西二郎
4,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,2006-01-19,NaN,取り消す,26831,小宮山展隆,NaN,39.0,安西二郎


In [3]:
#trial_dateを日付型に変更
df['trial_date'] = pd.to_datetime(df['trial_date'])

<h2>データ前処理</h2>

<h3>説明変数の前処理</h3>

In [4]:
#目的変数及びtrial_date以外のデータを説明変数として使う
exdf = df[df.columns[df.columns!=target]].copy()
#trial_dateとそれ以外に分ける
exdf1 = exdf[exdf.columns[exdf.columns!="trial_date"]]
exdf2 = exdf[["trial_date"]]

In [5]:
#データ確認
exdf1.head()

,id,case_name,case_type,court_name,trial_type,id.1,lawyer_name,office,id.2,judge_name
0,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,NaN,12013,高坂敬三,色川法律事務所,39.0,安西二郎
1,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,NaN,13064,永野周志,シード綜合法律事務所,39.0,安西二郎
2,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,NaN,20798,鳥山半六,色川法律事務所,39.0,安西二郎
3,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,NaN,24026,山崎徹,弁護士法人川越法律事務所,39.0,安西二郎
4,100,認定取消請求事件,特許権・行政訴訟,神戸地方裁判所,NaN,26831,小宮山展隆,NaN,39.0,安西二郎


In [6]:
#データ確認
exdf2.head()

,trial_date
0,2006-01-19
1,2006-01-19
2,2006-01-19
3,2006-01-19
4,2006-01-19


<h4>カテゴリ変数の前処理</h4>

In [7]:
#exdf1の各項目について、上位exnumの項目のみ採用し、残りはNaNにする。
#採用項目リスト
exList = pd.DataFrame(index=range(exnum), columns=exdf1.columns)

for c in exdf1.columns:
    
    #上位exnum個の要素を抽出
    tmp = exdf1[c].value_counts().iloc[:exnum]
    
    #一致しないものはNaNとする
    exdf1[c][exdf1[c].isin(tmp.index) == False] = pd.np.NaN
    
    #採用した項目を残しておく
    exList.loc[range(tmp.shape[0]), c] = tmp.sort_index().index
    

In [8]:
#データ確認
exdf1.head()

,id,case_name,case_type,court_name,trial_type,id.1,lawyer_name,office,id.2,judge_name
0,NaN,NaN,特許権・行政訴訟,神戸地方裁判所,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,特許権・行政訴訟,神戸地方裁判所,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,特許権・行政訴訟,神戸地方裁判所,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,特許権・行政訴訟,神戸地方裁判所,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,特許権・行政訴訟,神戸地方裁判所,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#データ確認
exList

,id,case_name,case_type,court_name,trial_type,id.1,lawyer_name,office,id.2,judge_name
0,1423,不当利得返還請求事件,その他・民事訴訟,大阪地方裁判所,判決,22005,三村量一,TMI総合法律事務所,320,三村量一
1,2511,不正競争行為差止等請求事件,その他・行政訴訟,大阪高等裁判所,NaN,22228,中嶋誠,はる総合法律事務所,426,中野哲弘
2,3103,不正競争行為差止等請求控訴事件,不正競争,新潟地方裁判所 三条支部,NaN,24194,中野哲,ビンガム・坂井・三村・相澤法律事務所外国法共同事業,1233,今井弘晃
3,4211,商標権侵害差止等請求事件,不正競争・民事訴訟,最高裁判所第一小法廷,NaN,24886,井上泰,ユアサハラ法律特許事務所,1354,塚原朋一
4,5073,審決取消,商標権,最高裁判所第三小法廷,NaN,25614,井章光,リープ法律事務所,1397,塩月秀平
5,6374,審決取消請求事件,商標権・民事訴訟,最高裁判所第二小法廷,NaN,26655,伊藤真,中村合同特許法律事務所,1623,大鷹一郎
6,6431,意匠権侵害差止等請求事件,商標権・行政訴訟,東京地方裁判所,NaN,29181,塚原朋一,中野哲法律事務所,1676,岡本岳
7,6432,損害賠償等請求事件,実用新案権,横浜地方裁判所,NaN,31327,塩月秀平,創英国際特許法律事務所,2479,嶋末和秀
8,8084,損害賠償等請求控訴事件,実用新案権・民事訴訟,水戸地方裁判所 龍ケ崎支部,NaN,37347,大野聖二,大野総合法律事務所,2722,東海林保
9,8085,損害賠償請求事件,実用新案権・行政訴訟,知的財産高等裁判所,NaN,39643,宍戸充,小松法律特許事務所,2811,柵木澄子


In [10]:
#各項目をバイナライズする
from sklearn import preprocessing as sp
leResult = []

for c in exdf1.columns:
    
    try:
        leResult.append(pd.DataFrame(index=exdf1.index, columns=c + "=" + exList[c].astype('str').dropna(), data=sp.label_binarize(exdf1[c].fillna(0), classes=exList[c].dropna())))
    except:
        leResult.append(pd.DataFrame(index=exdf1.index, columns=c + "=" + exList[c].dropna(), data=sp.label_binarize(exdf1[c].fillna('0'), classes=exList[c].dropna())))
    
leexdf1 = pd.concat(leResult, axis=1)

#データ確認
leexdf1.head()

,id=1423,id=2511,id=3103,id=4211,id=5073,id=6374,id=6431,id=6432,id=8084,id=8085,...,judge_name=森義之,judge_name=清水節,judge_name=滝澤孝臣,judge_name=田中孝一,judge_name=荒井章光,judge_name=西理,judge_name=西理香,judge_name=長谷川浩二,judge_name=飯村敏明,judge_name=高野輝久
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h4>カレンダ情報の前処理</h4>

In [11]:
#特徴量を抽出
exdf2['year'] = exdf2['trial_date'].dt.year
exdf2['month'] = exdf2['trial_date'].dt.month
exdf2['day'] = exdf2['trial_date'].dt.day
exdf2['week'] = exdf2['trial_date'].dt.week
exdf2['weekday'] = exdf2['trial_date'].dt.weekday

leexdf2 = exdf2.copy()
del leexdf2['trial_date']

#データ確認
leexdf2.head()

,year,month,day,week,weekday
0,2006,1,19,3,3
1,2006,1,19,3,3
2,2006,1,19,3,3
3,2006,1,19,3,3
4,2006,1,19,3,3


<h4>最終的な説明変数</h4>

In [12]:
exdf = pd.concat([leexdf1, leexdf2], axis=1)

#データ確認
exdf.head()

,id=1423,id=2511,id=3103,id=4211,id=5073,id=6374,id=6431,id=6432,id=8084,id=8085,...,judge_name=西理,judge_name=西理香,judge_name=長谷川浩二,judge_name=飯村敏明,judge_name=高野輝久,year,month,day,week,weekday
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2006,1,19,3,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2006,1,19,3,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2006,1,19,3,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2006,1,19,3,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2006,1,19,3,3


<h3>目的変数の前処理</h3>

In [13]:
#目的変数を抽出
tadf = df[[target]]

#データ確認
tadf.head()

,trial_result
0,取り消す
1,取り消す
2,取り消す
3,取り消す
4,取り消す


In [14]:
#シリアライズ
#項目を抽出
targetList = tadf[target].unique()
targetList.sort()

#ラベルエンコーダーでシリアライズ
le = sp.LabelEncoder()
le.fit(targetList)
tadf[target] = le.transform(tadf).tolist()

#データ確認
tadf.head()

,trial_result
0,5
1,5
2,5
3,5
4,5


<h3>前処理後のデータフレーム</h3>

In [15]:
#前処理後のデータフレーム
df2 = pd.concat([exdf, tadf], axis=1)
del exdf['year']

#データ確認
df2.head()

,id=1423,id=2511,id=3103,id=4211,id=5073,id=6374,id=6431,id=6432,id=8084,id=8085,...,judge_name=西理香,judge_name=長谷川浩二,judge_name=飯村敏明,judge_name=高野輝久,year,month,day,week,weekday,trial_result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2006,1,19,3,3,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2006,1,19,3,3,5
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2006,1,19,3,3,5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2006,1,19,3,3,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2006,1,19,3,3,5


<h2>機械学習によるモデル構築・精度検証</h2>

In [16]:
#ランダムフォレストを使う
from sklearn.ensemble import RandomForestClassifier

<h3>クラスの重みを設定</h3>

In [17]:
#データの量が少ないほど大きなweightにする。
weightData = pd.DataFrame(data=tadf['trial_result'].value_counts().sort_index().values, columns=['count'])
weightData['trial_result'] = le.classes_
weightData['weight'] = (1 / weightData['count'] * weightData['count'].max()).astype(int)
class_weight = dict(zip(weightData.index, weightData.weight))

#データ確認
weightData

,count,trial_result,weight
0,934,その他,101
1,70,一部棄却,1354
2,1150,一部認容,82
3,348,仮執行宣言,272
4,1895,却下,50
5,15606,取り消す,6
6,21,有罪,4515
7,94835,棄却,1
8,12,破棄,7902
9,22,破棄差戻し,4310


In [18]:
#ランダムフォレストのインスタンスを定義
clf = RandomForestClassifier(n_estimators=30, n_jobs=6, random_state=0, class_weight=class_weight, max_features=None)

<h3>クロスバリデーションによる精度の目途付け</h3>

In [19]:
#クロスバリデーションで精度を検証する
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, exdf.values, tadf.values[:, 0], cv=10)

In [20]:
scores

array([ 0.80578834,  0.85165025,  0.83753889,  0.8526616 ,  0.81254861,
        0.78710347,  0.78810409,  0.86772715,  0.78255231,  0.78263877])

In [21]:
print("平均精度", scores.mean()*100,"%")

平均精度 81.6831347184 %


<h3>実用例</h3>

In [22]:
#2015年までのデータでモデルを構築し、2016年以降のデータを予測する
lnDataindex = df2[df2['year'] < 2016].index
pdDataindex = df2[df2['year'] >= 2016].index

In [23]:
#学習
clf.fit(exdf.ix[lnDataindex].values, tadf.ix[lnDataindex].values[:, 0])

RandomForestClassifier(bootstrap=True,
            class_weight={0: 101, 1: 1354, 2: 82, 3: 272, 4: 50, 5: 6, 6: 4515, 7: 1, 8: 7902, 9: 4310, 10: 117},
            criterion='gini', max_depth=None, max_features=None,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=6,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [24]:
#正解
result = tadf.ix[pdDataindex].copy()
result.columns = ['true']
#予測
result['pred'] = clf.predict(exdf.ix[pdDataindex].values).tolist()

In [25]:
pd.pivot_table(result.reset_index(), index='pred', columns='true', aggfunc='count')

index                                               
true    0     1      2     3     4      5       7      10
pred                                                     
0      NaN   NaN    NaN   NaN   NaN    NaN     8.0    NaN
1      NaN   NaN    NaN   NaN   NaN    NaN     9.0    NaN
2      NaN   NaN    NaN   NaN  28.0    NaN   152.0   28.0
3      NaN   NaN    NaN   NaN   NaN    NaN    14.0    NaN
4     33.0   NaN    NaN   NaN   NaN    NaN   102.0   33.0
5      NaN   NaN    NaN   NaN   5.0  259.0   670.0    NaN
7     15.0  40.0  103.0  38.0  81.0  811.0  6729.0  168.0
10     NaN   NaN    NaN   NaN   NaN    NaN    42.0   19.0

予測結果として0, 1, 2 ,3, 4, 5, 7, 10が出力できるようになった。
<br>5, 7, 10に関しては予測が的中しているケースもある。

In [26]:
print("5は「" + le.classes_[5] + "」")
print("7は「" + le.classes_[7] + "」")
print("10は「" + le.classes_[10] + "」")

5は「取り消す」
7は「棄却」
10は「認容」


In [27]:
print("精度", result[result['true'] == result['pred']].shape[0] / result.shape[0]*100,"%")

精度 74.64578672632364 %
